In [2]:
def B1(thickness, TVL): # Calculates B factor using only 1 TVL. Important for PATIENT SCATTER
    '''
    param thickness: thickness of barrier (cm)
    param TVL: TVL (cm) where it is ALWAYS dependent on energy. Can also depend on the angle too
    '''
    B = 10**-(thickness/TVL)
    return B

def B2(thickness, TVL1, TVLe): # Calculates B factor using 2 TVLs. Important for LEAKAGE
    '''
    param thickness: thickness of barrier (cm)
    param TVL1: first TVL (cm)
    param TVLe: equilibrium TVL (cm)
    '''
    B = 10**-(1+((thickness-TVL1)/TVLe))
    return B

def H_ps_unshielded(W, d_secondary, d_scatter, a, F): # Calculates unshielded equivalent dose for PATIENT SCATTER
    '''
    param W: workload in a typical week (Gy/wk)
    param d_secondary: distance from isocenter to the point of interest (cm). Includes the 0.3 m
    param d_scatter: distance from target to isocenter. Usually 1 m
    param a: scatter fraction dependent on angle (deg) and beam energy (MV)
    param F: field size in cm^2
    '''
    H_ps_uns = (W * a * F) / (d_secondary**2 * d_scatter**2 * 400) * 10**3
    return H_ps_uns # (mSv/wk)

def H_ps_shielded(B1, H_ps_unshielded): # Determines SHIELDED equivalent dose for patient scatter
    H_ps_s = B1 * H_ps_unshielded
    return H_ps_s # (mSv/wk)

def H_leak_unshielded(W_L, d_leakage): # Calculates UNSHIELDED equivalent dose for LEAKAGE
    '''
    param W_L: leakage workload (Gy/wk)
    param d_leakage: distance from source of leakage to point of protection (cm)
    '''
    H_leak_uns = ((W_L*10**-3) / d_leakage**2) * 10**3 # 10**-3 is the included barriers from the head and 10**3 converts Sv to mSv
    return H_leak_uns # (mSv/wk)

def H_leak_shielded(B2, H_leak_unshielded): # Determines SHIELDED equivalent dose for LEAKAGE
    H_leak_s = B2 * H_leak_unshielded
    return H_leak_s # (mSv/wk)

# Wall A: Determining if the wall is meets design goal P

In [3]:
T_unattended_parking_lot = 1/40
x = H_ps_unshielded(1500, 8.473706, 1, 9.7725*10**-4, 1600)
y = B1(88.4515, 18.75)
H_shielded_ps = x*y*T_unattended_parking_lot
print('H patient scatter shielded including B and T factors')
print(f"{H_shielded_ps} mSv/wk")



H patient scatter shielded including B and T factors
3.913243037003771e-05 mSv/wk


In [4]:
x2 = H_leak_unshielded(1500*2, 8.473706) 
B_wallA_leak = B2(88.4515, 34, 29) # thickness is oblique thickness
H_shielded_leakage = x2*B_wallA_leak*T_unattended_parking_lot

print('H leakage shielded including B and T factors')
print(f"{H_shielded_leakage} mSv/wk")
print()
print('H total shielding for Wall A')
print(f"{H_shielded_leakage + H_shielded_ps} mSv/wk")

H leakage shielded including B and T factors
0.001384447682628563 mSv/wk

H total shielding for Wall A
0.0014235801129986007 mSv/wk


# Wall B Calculations

In [5]:
print("H total for wall B")
print(f"{H_ps_shielded(B1(91.44,17), H_ps_unshielded(1500,4.3,1,4.26*10**-4,1600)) * (1/2) + H_leak_shielded(B2(91.44,34,29), H_leak_unshielded(3000,4.3)) * (1/2)} mSv/wk")

H total for wall B
0.08510235801591565 mSv/wk


In [6]:
H_leak_shielded(B2(91.44,34,29), H_leak_unshielded(3000,4.3)) * (1/2)

0.08481344280106991

In [7]:
x = B1(76.2,17)
y = H_ps_unshielded(1500,4.3,1,4.26*10**-4,1600)
print(x*y*.5)

0.0022763602928343022


# Wall C Calculations

In [8]:
T_staffrest = 1/5
B_ps_wallC = B1(94.376,18.75)
H_ps_uns_wallC = H_ps_unshielded(1500,7.8027,1,9.805*10**-4,1600)

H_ps_shielded_wallC = B_ps_wallC * H_ps_uns_wallC * T_staffrest
print(f"Patient scatter for wall C: {H_ps_shielded_wallC} mSv/wk")

Patient scatter for wall C: 0.00017895851141378643 mSv/wk


In [9]:
print("Leakage for wall C")

H_leak_shielded_wallC = B2(94.376,34,29) * T_staffrest * H_leak_unshielded(3000,7.8027)
print(H_leak_shielded_wallC)

Leakage for wall C
0.008160756077777234


In [10]:
print("Total H shielded for wall C")
print(f"{H_ps_shielded_wallC + H_leak_shielded_wallC} mSv/wk")


Total H shielded for wall C
0.00833971458919102 mSv/wk


# Calculations for the ceiling

This assumes the realistic approach.

In [11]:
H_ps_uns_ceil = H_ps_unshielded(1500, 5.8859, 1, 9.805*10**-4,1600)
B_ps_ceil = B1(105.2348,18.75)
print(B_ps_ceil)
T_ceiling = 1/20

H_ps_shielded_ceil = H_ps_uns_ceil * B_ps_ceil * T_ceiling
print("H shielded patient scatter for ceiling")
print(f"{H_ps_shielded_ceil} mSv/wk")


2.440491690749476e-06
H shielded patient scatter for ceiling
2.0721463611475287e-05 mSv/wk


In [12]:
B_leak_ceil = B2(105.2348, 34, 29)
H_leak_uns_ceil = H_leak_unshielded(3000,5.8859)
H_leak_shielded_ceil = H_leak_uns_ceil * B_leak_ceil * T_ceiling
print("H shielded leakage for ceiling")
print(f"{H_leak_shielded_ceil} mSv/wk")


H shielded leakage for ceiling
0.0015138857000799905 mSv/wk


In [13]:
print("total H for ceiling (realistic case)")
print(f"{H_leak_shielded_ceil + H_ps_shielded_ceil} mSv/wk")

total H for ceiling (realistic case)
0.0015346071636914657 mSv/wk


For a worst case scenario,

In [14]:
asdf = H_ps_unshielded(1500, 5.8859, 1, 2.77*10**-3, 1600) * B1(105.23,26) * 1/20
fghj = H_leak_unshielded(3000, 5.5736) * B2(105.2348,34,29) * 1/20
print("total H for ceiling (worst case)")
print(f"{asdf + fghj} mSv/wk")

total H for ceiling (worst case)
0.003839425322276716 mSv/wk


# Problem 6

Below is for Point #1 (See Homework for diagram)

In [15]:
B_ps_wallZ = B1(45*2.54, 17)
print(B_ps_wallZ)
H_ps_uns_wallZ = H_ps_unshielded(1500, 8.562, 1, 4.26*10**-4, 1600)
print(H_ps_uns_wallZ)
H_ps_shielded_wallZ = B_ps_wallZ * H_ps_uns_wallZ * 1/5
print("H for patient scatter")
print(H_ps_shielded_wallZ)

1.890038227621723e-07
34.86666447317698
H for patient scatter
1.3179865744792944e-06


In [16]:
B_leak_wallZ = B2(45*2.54, 34, 29)
H_leak_uns_wallZ = H_leak_unshielded(3000, 8.562)
H_leak_shielded_wallZ = B_leak_wallZ * H_leak_uns_wallZ * 1/5
print("H for leakage")
print(H_leak_shielded_wallZ)

H for leakage
0.0013932700006327825


In [17]:
print("H total for wall Z")
print(H_ps_shielded_wallZ + H_leak_shielded_wallZ)

H total for wall Z
0.0013945879872072618


Below is Point #2 (See homework for diagram)

In [18]:
temp1 = H_ps_shielded(B1(98.923,23), H_ps_unshielded(1500,11.1152,1,1.39*10**-3,1600)) * (1/5)
print(temp1)

0.0006750897020726225


In [19]:
temp2 = H_leak_shielded(B2(98.923,34,29),H_leak_unshielded(3000,11.1152)) * (1/5)
print(temp2)

0.0028028028433049607


In [20]:
print("Total H at point 2")
print(temp1 + temp2)

Total H at point 2
0.003477892545377583


In [21]:
B2(98.923,34,29)

0.0005771329394576966

In [ ]:
T_unattended_parking_lot = 1/40
test = 88.4515
test2 = 7.802745
x = H_ps_unshielded(1500, 8.473706, 1, 2.77*10**-3, 1600)
y = B1(76.2, 26)
print('H patient scatter shielded including B and T factors')
print(f"{x*y*T_unattended_parking_lot} mSv/wk")

H patient scatter shielded including B and T factors
0.0067866431441343 mSv/wk


In [23]:
x2 = H_leak_unshielded(1500*2, 8.473706) 
B_wallA_leak = B2(test, 34, 29)
print('H leakage shielded including B and T factors')
print(f"{x2*B_wallA_leak*T_unattended_parking_lot} mSv/wk")

H leakage shielded including B and T factors
0.001384447682628563 mSv/wk
